## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [ ]:
from keras.datasets import cifar10
from keras.applications.resnet import ResNet50 
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Flatten, Dense, Dropout, Input, AveragePooling2D

# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')/ 255.
x_test = x_test.astype('float32')/ 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [ ]:
# write callback here
from keras.callbacks import LearningRateScheduler, TensorBoard, Callback, ReduceLROnPlateau
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.lr = []
        self.fig = plt.figure()
        
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(epoch)

        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('accuracy'))
        self.val_acc.append(logs.get('val_accuracy'))
        self.lr.append(logs.get('lr'))

        f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharex=True,figsize = (15,5))
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.set_title('epoch_loss')
        ax1.set(xlabel='Epoch', ylabel='loss')
        ax1.legend()

        ax2.plot(self.x, self.acc, label="acc")
        ax2.plot(self.x, self.val_acc, label="val_acc")
        ax2.set_title('epoch_accuracy')
        ax2.set(xlabel='Epoch', ylabel='accuracy')
        ax2.legend()
        
        ax3.plot(self.x, self.lr)
        ax3.set_title('Learning rate')
        ax3.set(xlabel='Epoch', ylabel='')

        plt.show();

class show_epoch(Callback):
    def on_epoch_end(self, epoch, logs={}): 
        print('Epoch: ',epoch+1)
        
display_epoch = show_epoch()

lr_scheduler = LearningRateScheduler(lr_schedule,
                                     verbose=0)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

TensorBoardColabCallback = TensorBoard(log_dir='./logs',
                                       histogram_freq=0,
                                       write_graph=False,
                                       write_images=False,
                                       embeddings_freq=0,
                                       embeddings_layer_names=None,
                                       embeddings_metadata=None)

plot_accval = PlotLearning()

callbacks = [lr_scheduler, TensorBoardColabCallback]

if TensorBoardColabCallback in callbacks:
    %load_ext tensorboard
    %tensorboard --logdir logs
else:
    print("Not using tensorboard.")

In [ ]:
# Hyperparamters
batch_size = 32 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 200 # 訓練整個資料集共 30個循環
data_augmentation = True

In [ ]:
# Model build here
net = ResNet50(include_top=False, weights=None, input_shape=x_train.shape[1:], classes=num_classes)
x = Flatten()(net.output)
out = Dense(units=10, activation='softmax')(x)
model = Model(net.input, out)
model.summary()

In [ ]:
# Training
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using data augmentation.')
    # Data augmentation
    datagen = ImageDataGenerator(
        horizontal_flip=True,
        zca_epsilon=1e-06,
        width_shift_range=0.1,
        height_shift_range=0.1)

    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs,
                        callbacks=callbacks,
                        verbose=2)

Using data augmentation.
Instructions for updating:
Please use Model.fit, which supports generators.
Learning rate:  0.001
Epoch 1/200
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1563/1563 - 204s - loss: 2.3217 - accuracy: 0.2796 - val_loss: 5.0117 - val_accuracy: 0.2388
Learning rate:  0.001
Epoch 2/200
1563/1563 - 202s - loss: 2.2327 - accuracy: 0.2774 - val_loss: 5.5346 - val_accuracy: 0.1870
Learning rate:  0.001
Epoch 3/200
1563/1563 - 201s - loss: 2.2458 - accuracy: 0.2645 - val_loss: 1.9642 - val_accuracy: 0.2966
Learning rate:  0.001
Epoch 4/200
1563/1563 - 202s - loss: 1.9170 - accuracy: 0.3513 - val_loss: 5.4953 - val_accuracy: 0.2410
Learning rate:  0.001
Epoch 5/200
1563/1563 - 201s - loss: 2.0800 - accuracy: 0.3114 - val_loss: 11.2669 - val_accuracy: 0.3236
Learning rate:  0.001
Epoch 6/200
1563/1563 - 202s - loss: 2.1368 - accuracy: 0.2761 - val_loss: 1.7202 - val_accuracy: 0.3851
Learning rate:  0.001
Epoch 7/200
1563/1563 - 201s - loss: 1.776